In [1]:
import pickle

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Setup Data
from utils.data import fires, FEATURE_NAMES
from utils.visualization import plot_decisions


%matplotlib inline

In [2]:
X, y_log = fires[FEATURE_NAMES], fires["log(area+1)"]

## Parameter Tuning

In [3]:
learning_rate_range = [1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1]
max_depth_range = [2, 5, 7, 9, 11, 13]
n_estimators_range = list(range(75, 175))

In [4]:
grid_search = GridSearchCV(
    xgb.XGBRegressor(),
    param_grid={"learning_rate": learning_rate_range,
                "max_depth": max_depth_range,
                "n_estimators": n_estimators_range},
    cv=10,
    n_jobs=-1,
    scoring="neg_mean_absolute_error",
)

In [ ]:
grid_search.fit(X, y_log)

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.best_score_

## Visualize Decisions

In [ ]:
plot_decisions(grid_search.best_estimator_, X, y_log)

## Saving the model

In [ ]:
with open("xgboost.b", "wb") as f:
    pickle.dump(grid_search.best_estimator_, f)